In [9]:
import pandas as pd
import numpy as np
import cv2
import os
DatasetFolderName='MathsCharacters'

In [12]:
print('Method to extract the images and appending them as an array')
def ExtractData(Path):
    Path=DatasetFolderName+'/'+Path
    CharacterArray=[]
    countOfImages=0

    for image in os.listdir(Path):
    #  while countOfImages<3000 and len(os.listdir(Path))>countOfImages:
        img = cv2.imread(os.path.join(Path,image),cv2.IMREAD_GRAYSCALE)
        img=~img
        if img is not None:
            ret,thresh=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
            ctrs,ret=cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
            cnt=sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
            Imagewidth=int(28)
            Imageheight=int(28)
            maxi=0
            for c in cnt:
                x,y,Imagewidth,Imageheight=cv2.boundingRect(c)
                maxi=max(Imagewidth*Imageheight,maxi)
                if maxi==Imagewidth*Imageheight:
                    x_max=x
                    y_max=y
                    w_max=Imagewidth
                    h_max=Imageheight
            im_crop= thresh[y_max:y_max+h_max+10, x_max:x_max+w_max+10]
            im_resize = cv2.resize(im_crop,(28,28))
            im_resize=np.reshape(im_resize,(784,1))
            CharacterArray.append(im_resize)
            countOfImages+=1
    #  else:
    #      break
    return CharacterArray



Method to extract the images and appending them as an array


In [13]:
print('printing the total classes and their samples')
CharacterSet=[]

for datafolder in os.listdir(DatasetFolderName):
    
        # to omit hidden folders sucha .Ds
    if(datafolder.startswith('.')):
        continue
    else :  
        datafolderPath=DatasetFolderName+'/'+datafolder
        print(str(len(os.listdir(datafolderPath)))+'->'+datafolder)

      
         

printing the total classes and their samples
3251->times
3737->9
6914->0
2909->7
25112->+
3118->6
26520->1
3068->8
33997->-
7396->4
10909->3
26141->2
3545->5


In [14]:
print('There is a lot of differnce in the number of images per each class')
# print('I tried to normalize that by doing shifts rotation but it didnt give better results so truncating all the images to 3000,so that each class contains images upto 3000 only')

There is a lot of differnce in the number of images per each class


In [15]:
c=0
import json
classdict = {}
 
for datafolder in os.listdir(DatasetFolderName):
  if datafolder.startswith('.'):
     continue
  print(datafolder+'->'+' Class Number' +str(c))
  classdict[c]=datafolder
  c+=1
json.dump(classdict, open("Mathscllasses.txt",'w'))


times-> Class Number0
9-> Class Number1
0-> Class Number2
7-> Class Number3
+-> Class Number4
6-> Class Number5
1-> Class Number6
8-> Class Number7
--> Class Number8
4-> Class Number9
3-> Class Number10
2-> Class Number11
5-> Class Number12


In [16]:
print('Saving the samples in the csv')
import os
#a CharacterSet variable to store all the characters with their labels
CharacterSet=[]
c=0
#to loop throuh the available folders inside our dataset
for datafolder in os.listdir(DatasetFolderName):
    
        # to omit hidden folders sucha .Ds
    if(datafolder.startswith('.')):
               continue
    else :     
               #while starting the CharacterSet extraction for first time we will copy the extracted CharacterSet
               if(len(CharacterSet)==0):
                  CharacterSet=ExtractData(datafolder)
                  for i in range(0,len(CharacterSet)):
                     CharacterSet[i]=np.append(CharacterSet[i],c)
                  print(str(len(CharacterSet))+'->'+datafolder)
               #character images appended recursively
               else:
                  Character=ExtractData(datafolder)
                  
               
                  for i in range(0,len(Character)):
                     Character[i]=np.append(Character[i],c)
                  CharacterSet=np.concatenate((CharacterSet,Character))
                  c+=1   
                  print(str(len(Character))+'->'+datafolder)
                  print('Total count'+str(len(CharacterSet)))


Saving the samples in the csv
3251->times
3737->9
Total count6988
6914->0
Total count13902
2909->7
Total count16811
25112->+
Total count41923
3118->6
Total count45041
26520->1
Total count71561
3068->8
Total count74629
33997->-
Total count108626
7396->4
Total count116022
10909->3
Total count126931
26141->2
Total count153072
3545->5
Total count156617


In [17]:
        
df=pd.DataFrame(CharacterSet,index=None)
df.to_csv('MathsCharactersDataset3.csv',index=False)
